## Requires the zipping of Data_10Q folder first into Data_10Q.zip

In [1]:
import gzip
from io import StringIO
import glob
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, NavigableString
from tqdm import tqdm
from zipfile import ZipFile
import unicodedata
import re
import os
import shutil

In [2]:
def get_zipped_filenames(file_name):
    files = []
    with ZipFile(file_name, "r") as zip:
        count = 0
        for info in zip.infolist():
            name = info.filename
            parts = name.split("/")
            if len(parts) == 3 and parts[-1] != "":
                year = parts[1]
                cik = parts[2].split("_")[0]
                unique_key = parts[2].split("_")[1]
                files.append([year, cik, unique_key, name])
    return files

def get_zipped_file(path):
    with ZipFile("../../Raw Data/Data_10Q.zip", "r") as zipped_file:
        zipped_file.extract(path)
        html = open(path).read()
        soup = BeautifulSoup(html)
        text = soup.get_text(" ")
        text = unicodedata.normalize("NFKD", text)
        text = text.replace("\t", " ")
        text = text.replace("/", " ")
        text = "".join([i if ord(i) < 128 else "" for i in text])
        text = " ".join(text.split())
        print(text)

In [3]:
# Item 2.Management’s Discussion and Analysis | Item 3. Quan|Qual
# Item 3.Quantitative and Qualitative Disclosures About Market Risk | Item 4. controls
# Item 1A. Risk Factors | Item 2.  Unregistered Sale
def extract_item(zipped_file, item_name, item_regex, save_path):
    with ZipFile(zipped_file, "r") as zipped_file:
        for i in tqdm(range(len(zipped_file.infolist()))):
            info = zipped_file.infolist()[i]
            if "html" in info.filename and "MACOS" not in info.filename:
                
                cik = info.filename.split("/")[1]
                date = info.filename.split("/")[2].split("_")[1][:-5]
                saving_path = save_path + item_name + "_" + cik + "_" + date + ".txt"
                
                if os.path.isfile(saving_path):
                    print(saving_path, "already exists")
                    continue
                
                print("Scraping ", info.filename)

                zipped_file.extract(info.filename)
                html = open(info.filename).read()
                soup = BeautifulSoup(html)
                text = soup.get_text(" ")
                text = unicodedata.normalize("NFKD", text)
                text = text.replace("\t", " ")
                text = text.replace("/", " ")
                text = "".join([i if ord(i) < 128 else "" for i in text])
                text = " ".join(text.split())
                original_text = text

                results = []
                matches = []
                matches = re.findall(item_regex, text, re.IGNORECASE)
                
                if matches:
                    while matches:
                        current_match = matches[0]
                        results.append(current_match)
                        text = text.replace(current_match[:20], " ", 1)
                        matches.remove(current_match)
                        matches.extend(re.findall(item_regex, text, re.IGNORECASE))\
                    
                    print("")
                    print("Saving as...", saving_path)
                    file_location = open(saving_path, "w")
                    file_location.write(results[-1])
                    file_location.close()
                    print("----SUCCESS----")
                else:
                    print("Failed")
                print("")
                print("")
                print("")
            if os.path.isdir("Data_10Q"):
                shutil.rmtree("Data_10Q")

In [4]:
zipped_file = "../../Raw Data/Data_10Q.zip"
item_name = "item1"
item_regex = "item[\s]*2[\s]*[-.:\s]*manage.*?item[\s]*3[\s]*[-.:\s]*(?:quan|qual)"
save_path = "../../Processed Data/10K10Q/Data_10Q/"
extract_item(zipped_file, item_name, item_regex, save_path)

  0%|                                                                                          | 0/910 [00:00<?, ?it/s]

../../Processed Data/10K10Q/Data_10Q/item1_18230_2010-07-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-07-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2012-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-05-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2019-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2014-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2011-05-09.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2016-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2019-08-01.txt already exists


  6%|████▍                                                                           | 51/910 [00:00<00:01, 493.29it/s]

../../Processed Data/10K10Q/Data_10Q/item1_18230_2014-08-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2012-11-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2014-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2010-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-10-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_201

  8%|██████▊                                                                         | 77/910 [00:00<00:02, 388.60it/s]

...

 97%|████████████████████████████████████████████████████████████████████████████▋  | 884/910 [00:05<00:00, 167.06it/s]

../../Processed Data/10K10Q/Data_10Q/item1_34088_2017-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2011-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2014-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2014-11-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2018-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2012-11-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2019-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2018-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_201

 99%|██████████████████████████████████████████████████████████████████████████████▏| 901/910 [00:05<00:00, 163.62it/s]

../../Processed Data/10K10Q/Data_10Q/item1_34088_2015-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2013-08-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-05-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2010-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2019-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2015-08-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2010-05-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2012-08-02.txt already exists


100%|███████████████████████████████████████████████████████████████████████████████| 910/910 [00:05<00:00, 166.87it/s]


In [5]:
zipped_file = "../../Raw Data/Data_10Q.zip"
item_name = "item3"
item_regex = "item[\s]*3[\s]*[-.:\s]*(?:quan|qual).*?item[\s]*4[\s]*[-.:\s]*control"
save_path = "../../Processed Data/10K10Q/Data_10Q/"
extract_item(zipped_file, item_name, item_regex, save_path)

  0%|                                                                                          | 0/910 [00:00<?, ?it/s]

../../Processed Data/10K10Q/Data_10Q/item3_18230_2010-07-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2015-07-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2012-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2015-05-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2019-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2013-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2014-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2011-05-09.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2016-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2019-08-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_201

  7%|█████▎                                                                          | 60/910 [00:00<00:01, 586.28it/s]

../../Processed Data/10K10Q/Data_10Q/item3_18230_2017-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2015-10-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2018-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2018-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2013-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2010-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2011-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2012-08-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2016-11-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_18230_2019-05-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_886982_20

  9%|███████▌                                                                        | 86/910 [00:00<00:01, 418.05it/s]

...

 97%|████████████████████████████████████████████████████████████████████████████▊  | 885/910 [04:19<00:00, 154.74it/s]

../../Processed Data/10K10Q/Data_10Q/item3_34088_2011-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2016-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2014-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2014-11-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2018-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2012-11-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2019-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2018-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2010-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_201

 99%|██████████████████████████████████████████████████████████████████████████████▎| 902/910 [04:19<00:00, 157.50it/s]

../../Processed Data/10K10Q/Data_10Q/item3_34088_2013-08-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2016-05-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2010-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2019-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2015-08-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2010-05-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item3_34088_2012-08-02.txt already exists


100%|████████████████████████████████████████████████████████████████████████████████| 910/910 [04:19<00:00,  3.51it/s]


In [6]:
zipped_file = "../../Raw Data/Data_10Q.zip"
item_name = "item1a"
# Item 1A. Risk Factors | Item 2.  Unregistered Sale
item_regex = "item[\s]*1[\s]*a*[-.:\s]*risk.*?item[\s]*2[-.:\s]*unregis"
save_path = "../../Processed Data/10K10Q/Data_10Q/"
extract_item(zipped_file, item_name, item_regex, save_path)
# 886982, 51143, 200406, 34088 all don't have, already checked manually

  0%|                                                                                          | 0/910 [00:00<?, ?it/s]

../../Processed Data/10K10Q/Data_10Q/item1a_18230_2010-07-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2015-07-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2012-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2015-05-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2019-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2013-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2014-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2011-05-09.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2016-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2019-08-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/ite

  7%|█████▎                                                                          | 60/910 [00:00<00:01, 575.37it/s]

../../Processed Data/10K10Q/Data_10Q/item1a_18230_2017-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2015-10-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2018-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2018-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2013-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2010-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2011-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2012-08-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2016-11-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1a_18230_2019-05-06.txt already exists
Scraping  Data_10Q/886982/886982_2014-11

  7%|█████▎                                                                          | 60/910 [00:20<00:01, 575.37it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████▊| 908/910 [08:41<00:03,  1.81s/it]

Scraping  Data_10Q/34088/34088_2010-05-06.html
Failed





100%|███████████████████████████████████████████████████████████████████████████████▉| 909/910 [08:43<00:01,  1.78s/it]

Scraping  Data_10Q/34088/34088_2012-08-02.html
Failed





100%|████████████████████████████████████████████████████████████████████████████████| 910/910 [08:47<00:00,  2.41s/it]


In [7]:
zipped_file = "../../Raw Data/Data_10Q.zip"
item_name = "item1"
item_regex = "item[\s]*2[\s]*[-.:\s]*manage.*?item[\s]*4[\s]*[-.:\s]*contr"
save_path = "../../Processed Data/10K10Q/Data_10Q/"
extract_item(zipped_file, item_name, item_regex, save_path)

  0%|                                                                                          | 0/910 [00:00<?, ?it/s]

../../Processed Data/10K10Q/Data_10Q/item1_18230_2010-07-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-07-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2012-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-05-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2019-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2014-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2011-05-09.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2016-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2019-08-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_201

  6%|████▉                                                                           | 56/910 [00:00<00:01, 550.49it/s]

../../Processed Data/10K10Q/Data_10Q/item1_18230_2010-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-10-31.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2017-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2015-10-30.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2018-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2013-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2010-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2011-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_18230_201

  8%|██████▊                                                                         | 77/910 [00:00<00:02, 367.00it/s]

...

 97%|████████████████████████████████████████████████████████████████████████████▋  | 884/910 [00:05<00:00, 157.33it/s]

../../Processed Data/10K10Q/Data_10Q/item1_34088_2017-11-01.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2011-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2011-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-11-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-08-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2014-05-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2014-11-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2018-05-03.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2012-11-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2013-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2019-08-07.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2018-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_201

 99%|██████████████████████████████████████████████████████████████████████████████▏| 901/910 [00:05<00:00, 154.48it/s]

../../Processed Data/10K10Q/Data_10Q/item1_34088_2015-11-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2013-08-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2017-08-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2016-05-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2010-08-04.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2019-05-02.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2015-08-05.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2010-05-06.txt already exists
../../Processed Data/10K10Q/Data_10Q/item1_34088_2012-08-02.txt already exists


100%|███████████████████████████████████████████████████████████████████████████████| 910/910 [00:05<00:00, 159.36it/s]
